# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [10]:
import heapq
from math import sqrt
import numpy as np

class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [11]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [12]:
def solution(node):
    action_list = []
    while node.parent is not None:
        action_list.append(node.action)
        node = node.parent
    print("Len of solution: ", len(action_list))
    return action_list[::-1]


class Node2:
    def __init__(self, problem, state, parent, action, path_cost, heuristic_cost):
        self.problem = problem
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost
        self.f_cost = self.path_cost + self.heuristic_cost

    # if heapq starts to compare node with node, it means, that nodes are identical
    def __lt__(self, other):
        return False


def astar(problem):
    initial_node = Node2(problem=problem, state=problem.initial_state, parent=None, action=None, path_cost=0,
                         heuristic_cost=problem.heuristic(problem.initial_state))
    explored_states = []
    priority_queue = [(initial_node.f_cost, (0, problem.initial_state, initial_node))]
    while priority_queue:
        priority, data_set = heapq.heappop(priority_queue)
        current_path_cost, current_state, node = data_set
        if current_state in explored_states:
            continue
        explored_states.append(current_state)
        if problem.is_goal(current_state):
            return node
        for action in problem.available_actions(state=current_state):
            next_state = problem.do_action(current_state, action)
            if next_state not in explored_states:
                child_node = Node2(problem=problem, state=next_state, parent=node, action=action,
                                   path_cost=problem.action_cost(current_state, action) + current_path_cost,
                                   heuristic_cost=problem.heuristic(next_state))
                new_element = (child_node.f_cost, (child_node.path_cost, next_state, child_node))
                heapq.heappush(priority_queue, new_element)

Now lets test your code in the vacuum world!

In [13]:
%timeit final_node = astar(VacuumProblem())
print(solution(final_node))

16.5 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Len of solution:  3
['Suck', 'Right', 'Suck']


## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [14]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
%timeit final_node = astar(VacuumProblem1())
print(solution(final_node))

19.8 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Len of solution:  3
['Suck', 'Right', 'Suck']


And another in which heuristic grossly overestimates the cost.

In [15]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
%timeit final_node = astar(VacuumProblem2())
print(solution(final_node))

11 µs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Len of solution:  3
['Suck', 'Right', 'Suck']


**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

Although last heuristic appears to be the best, it is not admissible, because it doesn't satisfy properties of the heuristic. (It overestimates distance to the goal). It is the best, because this problem is solved in the fastest way, if the first possible move is "SUCK", and VacuumProblem2 promotes the most this move.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [16]:
class PuzzleProblem():
    def __init__(self, init_state, goal_state):
        self.init_state = init_state
        self.goal_state = goal_state
        self.how_many_to_count = 0

    # Agent should work on immutables
    def convert_to_tuple(self, state):
        this_tuple = tuple(list([tuple(list(state[0])), tuple(list(state[1])), tuple(list(state[2]))]))
        return this_tuple

    def convert_to_np_array(self, state):
        a_list = []
        for x in state:
            a_list.append(list(x))
        array = np.array(a_list)
        return array

    @property
    def initial_state(self):
        return self.convert_to_tuple(self.init_state)

    def get_position_of_0(self, state):
        position = np.where(state == 0)
        return position[0][0], position[1][0]

    def available_actions(self, state):
        state = self.convert_to_np_array(state)

        a, b = self.get_position_of_0(state)
        available_actions_ = list()
        if b != 0:
            available_actions_.append('Left')
        if b != 2:
            available_actions_.append('Right')
        if a != 0:
            available_actions_.append('Up')
        if a != 2:
            available_actions_.append('Down')

        return available_actions_

    def do_action(self, state, action):
        state = self.convert_to_np_array(state)

        a, b = self.get_position_of_0(state)
        new_state = np.copy(state)
        if action == 'Left':
            if b == 0:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a][b - 1] = new_state[a][b - 1], new_state[a][b]
        elif action == 'Right':
            if b == 2:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a][b + 1] = new_state[a][b + 1], new_state[a][b]
        elif action == 'Up':
            if a == 0:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a - 1][b] = new_state[a - 1][b], new_state[a][b]
        elif action == 'Down':
            if a == 2:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a + 1][b] = new_state[a + 1][b], new_state[a][b]
        else:
            raise Exception('Invalid action.')

        new_state = self.convert_to_tuple(new_state)
        return new_state

    def is_goal(self, state) -> bool:
        state = self.convert_to_np_array(state)
        return (state == self.goal_state).all()

    def action_cost(self, state, action) -> float:
        return 1

    class Number:
        def __init__(self, state, value):
            position = np.where(state == value)
            self.y = position[0][0]
            self.x = position[1][0]
            self.value = value
            
    def euklidean_distances_of_elements(self, state) -> float:
        state = self.convert_to_np_array(state)
        current_elements = [PuzzleProblem.Number(state=state, value=x) for x in range(self.how_many_to_count, 9)]
        goal_elements = [PuzzleProblem.Number(state=self.goal_state, value=x) for x in
                         range(self.how_many_to_count, 9)]
        distances = [sqrt((current_element.x - goal_element.x) ** 2) + sqrt((current_element.y - goal_element.y) ** 2)
                     for
                     current_element, goal_element in zip(current_elements, goal_elements)]
        return sum(distances)

    def manhatan_distances_of_elements(self, state) -> float:
        state = self.convert_to_np_array(state)
        current_elements = [PuzzleProblem.Number(state=state, value=x) for x in range(self.how_many_to_count, 9)]
        goal_elements = [PuzzleProblem.Number(state=self.goal_state, value=x) for x in
                         range(self.how_many_to_count, 9)]
        distances = [abs(current_element.x - goal_element.x) + abs(current_element.y - goal_element.y) for
                     current_element, goal_element in zip(current_elements, goal_elements)]
        return sum(distances)
    

    def minkowski_distance(self, state) -> float:
        p = 1.5
        state = self.convert_to_np_array(state)
        current_elements = [PuzzleProblem.Number(state=state, value=x) for x in range(self.how_many_to_count, 9)]
        goal_elements = [PuzzleProblem.Number(state=self.goal_state, value=x) for x in
                         range(self.how_many_to_count, 9)]
        distances = [
            ((abs(current_element.x - goal_element.x)) ** p + (abs(current_element.y - goal_element.y)) ** p) ** (1 / p)
            for
            current_element, goal_element in zip(current_elements, goal_elements)]
        return sum(distances)

    def heuristic(self, state) -> float:
        return self.euklidean_distances_of_elements(state)
problem = PuzzleProblem(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
final_node = astar(problem)
print(solution(final_node))

Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


In [17]:
problem = PuzzleProblem(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
%timeit final_node = astar(problem)
print(solution(final_node))

2.08 s ± 6.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


**Prove that this heuristic is admissible.**

Choosen heurystic - euklidean distance of every element from current state to goal state. 
Euklidean distance will be never longer than real real distance.

In [18]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:
        return self.manhatan_distances_of_elements(state)

In [23]:
problem = PuzzleProblem1(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
%timeit final_node = astar(problem)
print(solution(final_node))

2.07 s ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


**Prove that this heuristic is admissible.**

Choosen heurystic - manhatan distance of every element from current state to goal state. 
Any action can reduce the distance of any element to goal by at most one, thus heurystic is going to be always smaller than real distance.

In [20]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        return self.minkowski_distance(state)

In [21]:
problem = PuzzleProblem2(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
%timeit final_node = astar(problem)
print(solution(final_node))

4.67 s ± 9.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']


**Prove that this heuristic is admissible.**

Choosen heurystic - minkowski distance of every element from current state to goal state. Minkowski distance with p=1.5 is always smaller than manhatan distance, and thus always smaller than the real distance.

Run your heuristics on the given problem.

In [22]:
problem = PuzzleProblem(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
final_node = astar(problem)
print(solution(final_node))
problem1 = PuzzleProblem1(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
final_node1 = astar(problem1)
print(solution(final_node1))
problem2 = PuzzleProblem2(np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]]), np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
final_node2 = astar(problem2)
print(solution(final_node2))
print()
print("Is plan0==plan1?", solution(final_node) == solution(final_node1))
print("Is plan0==plan2?", solution(final_node) == solution(final_node2))
print("Is plan1==plan2?", solution(final_node1) == solution(final_node2))

Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']
Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']
Len of solution:  26
['Left', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up']

Len of solution:  26
Len of solution:  26
Is plan0==plan1? True
Len of solution:  26
Len of solution:  26
Is plan0==plan2? True
Len of solution:  26
Len of solution:  26
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

Manhatan distance appears to be the best heuristic, it returns an estimation of the distance to the goal with the least amount of calculation. It projects the real distance in the most credible way.


------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.